https://www.youtube.com/watch?v=quh7z1q7-uc

40:59

In [1]:
import torch
import torch.nn as nn
import tiktoken
import regex as re

# Initial Setup

In [2]:
with open('../data/datasets/the_verdict.txt', 'r') as f:
    entire_document = f.read()

len(entire_document)

20479

In [3]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', entire_document)
len(preprocessed)

#Includes Spaces with the preprocessing.
preprocessed

['I',
 ' ',
 'HAD',
 ' ',
 'always',
 ' ',
 'thought',
 ' ',
 'Jack',
 ' ',
 'Gisburn',
 ' ',
 'rather',
 ' ',
 'a',
 ' ',
 'cheap',
 ' ',
 'genius',
 '--',
 'though',
 ' ',
 'a',
 ' ',
 'good',
 ' ',
 'fellow',
 ' ',
 'enough',
 '--',
 'so',
 ' ',
 'it',
 ' ',
 'was',
 ' ',
 'no',
 ' ',
 'great',
 ' ',
 'surprise',
 ' ',
 'to',
 ' ',
 'me',
 ' ',
 'to',
 ' ',
 'hear',
 ' ',
 'that',
 ',',
 '',
 ' ',
 'in',
 ' ',
 'the',
 ' ',
 'height',
 ' ',
 'of',
 ' ',
 'his',
 ' ',
 'glory',
 ',',
 '',
 ' ',
 'he',
 ' ',
 'had',
 ' ',
 'dropped',
 ' ',
 'his',
 ' ',
 'painting',
 ',',
 '',
 ' ',
 'married',
 ' ',
 'a',
 ' ',
 'rich',
 ' ',
 'widow',
 ',',
 '',
 ' ',
 'and',
 ' ',
 'established',
 ' ',
 'himself',
 ' ',
 'in',
 ' ',
 'a',
 ' ',
 'villa',
 ' ',
 'on',
 ' ',
 'the',
 ' ',
 'Riviera',
 '.',
 '',
 ' ',
 '',
 '(',
 'Though',
 ' ',
 'I',
 ' ',
 'rather',
 ' ',
 'thought',
 ' ',
 'it',
 ' ',
 'would',
 ' ',
 'have',
 ' ',
 'been',
 ' ',
 'Rome',
 ' ',
 'or',
 ' ',
 'Florence',
 '.',
 '',


In [4]:
#Removing Duplicates
unique_tokens = sorted(set(preprocessed))

#All the unique tokens found within the text -- No duplicates.
#unique_tokens


vocab_size = len(unique_tokens)

#print(vocab_size)

#Entire size of the vocab document tokens found within the text -- No duplicates.
vocab_size

1133

In [5]:
for i in enumerate(unique_tokens):
    print(i)

(0, '')
(1, '\n')
(2, ' ')
(3, '!')
(4, '"')
(5, "'")
(6, '(')
(7, ')')
(8, ',')
(9, '--')
(10, '.')
(11, ':')
(12, ';')
(13, '?')
(14, 'A')
(15, 'Ah')
(16, 'Among')
(17, 'And')
(18, 'Are')
(19, 'Arrt')
(20, 'As')
(21, 'At')
(22, 'Be')
(23, 'Begin')
(24, 'Burlington')
(25, 'But')
(26, 'By')
(27, 'Carlo')
(28, 'Chicago')
(29, 'Claude')
(30, 'Come')
(31, 'Croft')
(32, 'Destroyed')
(33, 'Devonshire')
(34, 'Don')
(35, 'Dubarry')
(36, 'Emperors')
(37, 'Florence')
(38, 'For')
(39, 'Gallery')
(40, 'Gideon')
(41, 'Gisburn')
(42, 'Gisburns')
(43, 'Grafton')
(44, 'Greek')
(45, 'Grindle')
(46, 'Grindles')
(47, 'HAD')
(48, 'Had')
(49, 'Hang')
(50, 'Has')
(51, 'He')
(52, 'Her')
(53, 'Hermia')
(54, 'His')
(55, 'How')
(56, 'I')
(57, 'If')
(58, 'In')
(59, 'It')
(60, 'Jack')
(61, 'Jove')
(62, 'Just')
(63, 'Lord')
(64, 'Made')
(65, 'Miss')
(66, 'Money')
(67, 'Monte')
(68, 'Moon-dancers')
(69, 'Mr')
(70, 'Mrs')
(71, 'My')
(72, 'Never')
(73, 'No')
(74, 'Now')
(75, 'Nutley')
(76, 'Of')
(77, 'Oh')
(78, 'On'

In [6]:
vocab = {token:integer for integer, token in enumerate(unique_tokens)}

In [7]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('', 0)
('\n', 1)
(' ', 2)
('!', 3)
('"', 4)
("'", 5)
('(', 6)
(')', 7)
(',', 8)
('--', 9)
('.', 10)
(':', 11)
(';', 12)
('?', 13)
('A', 14)
('Ah', 15)
('Among', 16)
('And', 17)
('Are', 18)
('Arrt', 19)
('As', 20)
('At', 21)
('Be', 22)
('Begin', 23)
('Burlington', 24)
('But', 25)
('By', 26)
('Carlo', 27)
('Chicago', 28)
('Claude', 29)
('Come', 30)
('Croft', 31)
('Destroyed', 32)
('Devonshire', 33)
('Don', 34)
('Dubarry', 35)
('Emperors', 36)
('Florence', 37)
('For', 38)
('Gallery', 39)
('Gideon', 40)
('Gisburn', 41)
('Gisburns', 42)
('Grafton', 43)
('Greek', 44)
('Grindle', 45)
('Grindles', 46)
('HAD', 47)
('Had', 48)
('Hang', 49)
('Has', 50)


In [8]:
"""Very Basic Class on how to Tokenize a document."""
class SimpleTokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i]for i in ids])
        #Replaces spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [9]:
"""Class Works as Intended"""
#tokenizer = SimpleTokenizer(vocab)
#tokenizer.encode(doc)

'Class Works as Intended'

# Tiktoken

In [10]:
#Tiktoken library
#GPT2 Model is the same as the one define a few cells down
tokenizer = tiktoken.get_encoding('gpt2')

In [11]:
integers = tokenizer.encode(entire_document, allowed_special={"<|endoftext|>"})
integers

[40,
 367,
 2885,
 1464,
 1807,
 3619,
 402,
 271,
 10899,
 2138,
 257,
 7026,
 15632,
 438,
 2016,
 257,
 922,
 5891,
 1576,
 438,
 568,
 340,
 373,
 645,
 1049,
 5975,
 284,
 502,
 284,
 3285,
 326,
 11,
 287,
 262,
 6001,
 286,
 465,
 13476,
 11,
 339,
 550,
 5710,
 465,
 12036,
 11,
 6405,
 257,
 5527,
 27075,
 11,
 290,
 4920,
 2241,
 287,
 257,
 4489,
 64,
 319,
 262,
 34686,
 41976,
 13,
 357,
 10915,
 314,
 2138,
 1807,
 340,
 561,
 423,
 587,
 10598,
 393,
 28537,
 2014,
 198,
 198,
 1,
 464,
 6001,
 286,
 465,
 13476,
 1,
 438,
 5562,
 373,
 644,
 262,
 1466,
 1444,
 340,
 13,
 314,
 460,
 3285,
 9074,
 13,
 46606,
 536,
 5469,
 438,
 14363,
 938,
 4842,
 1650,
 353,
 438,
 2934,
 489,
 3255,
 465,
 48422,
 540,
 450,
 67,
 3299,
 13,
 366,
 5189,
 1781,
 340,
 338,
 1016,
 284,
 3758,
 262,
 1988,
 286,
 616,
 4286,
 705,
 1014,
 510,
 26,
 475,
 314,
 836,
 470,
 892,
 286,
 326,
 11,
 1770,
 13,
 8759,
 2763,
 438,
 1169,
 2994,
 284,
 943,
 17034,
 318,
 477,
 314,
 892,


In [12]:
strings = tokenizer.decode(integers)

print(strings)

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)

"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it's going to send the value of my picture 'way up; but I don't think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing's lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn's "Moon-dancers" to say, with tears in her eyes: "We shall not look upon its like again"?

Well!--even through th

In [13]:
from dataloader import create_dataloader_v1

loader = create_dataloader_v1(entire_document, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iterator = iter(loader)
input, targets = next(data_iterator)
print("Inputs:\n", input)
print("Targets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
